In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [2]:
psf = 'inp_files/da.psf'
pdb = 'inp_files/smd_ini.pdb'
prm = 'inp_files/par_all27_prot_lipid_cmap.prm'

In [232]:
def com_vel(v, mas):
    return (np.sum(mas * v, axis = 0) / np.sum(mas))

In [242]:
def KE(v, mas):
    return ((0.5 / na) * np.sum(mas * v**2))

In [244]:
alpha = (3 * kb * T) / (2 * KE(initial_velocities, mass))

In [246]:
initial_velocities *= alpha

In [247]:
initial_velocities -= com_vel(initial_velocities, mass)

In [3]:
import parmed

In [4]:
from parmed.charmm import CharmmParameterSet

In [5]:
prm = CharmmParameterSet("inp_files/par_all27_prot_lipid_cmap.prm")

/home/shaunak/anaconda3/envs/MD/lib/python3.6/site-packages/parmed/charmm/parameters.py:747: ParameterWarning: Atom type C not present in AtomType list
  warnings.warn('Atom type %s not present in AtomType list' % key, ParameterWarning)


In [9]:
from parmed.formats.registry import load_file

In [21]:
psf = load_file('inp_files/da.psf')

# Alternatively this is also okay
# from parmed.charmm import CharmmPsfFile
# psf = CharmmPsfFile('inp_files/da.psf')
psf.bonds

TrackedList([
	<Bond <Atom N [0]; In ALA 0>--<Atom CA [3]; In ALA 0>; type=None>
	<Bond <Atom HT2 [1]; In ALA 0>--<Atom N [0]; In ALA 0>; type=None>
	<Bond <Atom HT3 [2]; In ALA 0>--<Atom N [0]; In ALA 0>; type=None>
	<Bond <Atom CA [3]; In ALA 0>--<Atom HA [4]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom CA [3]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom HB1 [6]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom HB2 [7]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom HB3 [8]; In ALA 0>; type=None>
	<Bond <Atom C [9]; In ALA 0>--<Atom CA [3]; In ALA 0>; type=None>
	<Bond <Atom C [9]; In ALA 0>--<Atom N [11]; In ALA 1>; type=None>
	<Bond <Atom O [10]; In ALA 0>--<Atom C [9]; In ALA 0>; type=None>
	<Bond <Atom N [11]; In ALA 1>--<Atom HN [12]; In ALA 1>; type=None>
	<Bond <Atom N [11]; In ALA 1>--<Atom CA [13]; In ALA 1>; type=None>
	<Bond <Atom CA [13]; In ALA 1>--<Atom HA [14]; In ALA 1>; type=None>
	<Bond <Atom CB [15]; In ALA 1>

In [22]:
pdb = load_file('inp_files/smd_ini.pdb')

In [23]:
pdb.coordinates

array([[ 1.6600e-01,  2.6700e-01, -3.0400e-01],
       [-5.4400e-01,  1.8300e-01,  4.3700e-01],
       [ 9.4900e-01,  8.1700e-01,  1.8400e-01],
       [ 7.6700e-01, -1.1160e+00, -5.0600e-01],
       [-1.1000e-02, -1.8060e+00, -5.0800e-01],
       [ 1.3150e+00, -1.2430e+00, -1.9140e+00],
       [ 1.6520e+00, -2.2170e+00, -2.2730e+00],
       [ 4.4500e-01, -1.0150e+00, -2.5850e+00],
       [ 2.0220e+00, -4.8000e-01, -2.1480e+00],
       [ 1.8770e+00, -1.4790e+00,  5.1900e-01],
       [ 2.2040e+00, -6.5500e-01,  1.3490e+00],
       [ 2.5630e+00, -2.6420e+00,  2.9400e-01],
       [ 2.3540e+00, -3.2190e+00, -4.8800e-01],
       [ 3.7350e+00, -3.1400e+00,  1.0030e+00],
       [ 3.5390e+00, -4.1940e+00,  1.2370e+00],
       [ 4.9550e+00, -3.0560e+00,  1.1600e-01],
       [ 5.0750e+00, -2.0040e+00, -1.3000e-01],
       [ 5.9060e+00, -3.4220e+00,  4.9700e-01],
       [ 4.7380e+00, -3.5340e+00, -9.1800e-01],
       [ 4.0730e+00, -2.6180e+00,  2.4280e+00],
       [ 3.3890e+00, -2.8910e+00,  3.407

In [81]:
from copy import deepcopy

class Energies:
    def __init__(self, psf, pdb, prm):
        self.psf = psf
        self.pdb = pdb
        self.prm = prm
        
        ## initializing bond types and coordinates
        coords = pdb.coordinates
        bonds = psf.bonds
        angles = psf.angles
        dihedrals = psf.dihedrals
        
        for bond in bonds:
            ## bond types
            tid1 = bond.atom1.type
            tid2 = bond.atom2.type
            
            t = (tid1, tid2)
            bond.type = prm.bond_types[t]
    
            ## coordinates
            b1 = bond.atom1
            b1.xx, b1.xy, b1.xz = coords[b1.idx]
            
            b2 = bond.atom2
            b2.xx, b2.xy, b2.xz = coords[b2.idx]
            
        ## initializing angle types
        for angle in angles:
            a1 = angle.atom1.type
            a2 = angle.atom2.type
            a3 = angle.atom3.type
            
            t = (a1, a2, a3)
            
            angle.type = prm.angle_types[t]
            a = angle.measure()
            a0 = angle.type
            
        ## dihedral types
#         for d in prm.dihedral_types:
#             print(d)
        print('------------------------------------')
        for dihedral in dihedrals:
            a1 = dihedral.atom1.type
            a2 = dihedral.atom2.type
            a3 = dihedral.atom3.type
            a4 = dihedral.atom4.type
            
            t = (a1, a2, a3, a4)
            h = t[::-1]
#             print(prm.dihedral_types)
            print(t in prm.dihedral_types or h in prm.dihedral_types)
#             print(prm.dihedral_types[t])
            
        
    
    def bond(self):
        bonds = psf.bonds
        s = 0
        for bond in bonds:
            r1 = pdb.coordinates[bond.atom1.idx]
            r2 = pdb.coordinates[bond.atom2.idx]
            k = bond.type.k
            dist = np.linalg.norm(r1 - r2)
            
            r0 = bond.type.req
            cust_energy = k * ((dist - r0)**2)
            s += cust_energy
            
        return s
    
    def angle(self):
        s = 0
        for angle in psf.angles:
            theta = angle.measure()
            theta0 = angle.type.theteq
            k = angle.type.k
            theta_diff = (np.pi * (theta - theta0)) / 180
            en = k * (theta_diff ** 2)
            print(en, angle.energy())
            s += en
        return s

#     def dihedral_angle(self):
#         s = 0
#         for 
        
        
            


In [82]:
h = Energies(psf, pdb, prm)
# h.dihedral_angle()

------------------------------------
False
False
False
True
True
False
False
False
False
False
False
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
True
True
True
True
True
Fals